# Load data and set inventory number as row index

In [109]:
import pandas as pd
import numpy as np
import math
from time import sleep
import requests
import os
working_directory = os.getcwd()
print(working_directory)

birthDateLimit = '1800'
deathDateLimit = '1926'
copyright_applies_date = '1926'

def get_list_from_csv(filename):
    line_list = []
    with open(filename, 'rt', encoding='utf-8') as file_object:
        for one_line in file_object:
            line_list.append(one_line[:-1]) # remove dangling newline from end of line
    return line_list

styles = get_list_from_csv('style_period.csv')

def generate_header_dictionary(accept_media_type):
    user_agent_header = 'VanderBot/1.7.1 (https://github.com/HeardLibrary/linked-data/tree/master/vanderbot; mailto:steve.baskauf@vanderbilt.edu)'
    request_header_dictionary = {
        'Accept' : accept_media_type,
        'Content-Type': 'application/sparql-query',
        'User-Agent': user_agent_header
    }
    return request_header_dictionary

accept_media_type = 'application/json'
requestheader = generate_header_dictionary(accept_media_type)
endpoint = 'https://query.wikidata.org/sparql'

def work_too_old(date_before, work):
    too_old = False
    if not pd.isnull(work['inception_latest_date_val']):
        #print(type(work['inception_latest_date_val'] ))
        #print(work['inception_latest_date_val'] )
        if work['inception_latest_date_val'] < date_before:
            #print('late_date',work['inception_latest_date_val'] )
            too_old = True
    else:
        if not pd.isnull(work['inception_val']):
            if work['inception_val'] < date_before:
                #print('inception_val',work['inception_val'] )
                too_old = True
    return too_old

def old_style_period(styles, work):
    found = False
    if not pd.isnull(work['style_period']):
        for style in styles:
            if style == work['style_period']:
                found = True
    return found

# Query Wikidata Query Service, returns the value of a single property for an item by Q ID
def single_property_values_for_item(pid, qid):
    query = '''
select distinct ?object where {
wd:'''+ qid + ''' wdt:''' + pid + ''' ?object.
}'''
    #print(query)
    r = requests.post(endpoint, data=query.encode('utf-8'), headers=requestheader)
    results_list = []
    try:
    #if 1==1: # replace try: to let errors occur, also comment out the except: clause
        data = r.json()
        #print(data)
        statements = data['results']['bindings']
        if len(statements) > 0: # if no results, the list remains empty
            for statement in statements:
                result_value = statement['object']['value']
                results_list.append(result_value)
    except:
        results_list = [r.text]

    # delay by some amount (quarter second default) to avoid hitting the SPARQL endpoint too rapidly
    sleep(0.1)
    return results_list

def person_too_old(birthDateLimit, qId):
    screen = False
    birthDateList = single_property_values_for_item('P569', qId)
    if len(birthDateList) == 0:
        #print('No birth date given.')
    else:
        birthDate = birthDateList[0][0:10] # all dates are converted to xsd:dateTime and will have a y-m-d date
        #print('This person was born in ', birthDate)
        if birthDate < birthDateLimit:
            # if the person was born a long time ago, don't retrieve other stuff
            #print('*** Too old !')
            screen = True
    return screen

def dead(deathDateLimit, qId):
    screen = False
    deathDateList = single_property_values_for_item('P570', qId)
    if len(deathDateList) == 0:
        #print('No death date given.')
    else:
        deathDate = deathDateList[0][0:10] # all dates are converted to xsd:dateTime and will have a y-m-d date
        #print('This person died in ', deathDate)
        if deathDate < deathDateLimit:
            # if the person died a long time ago, don't retrieve other stuff
            screen = True
            #print('*** Died too long ago !')
    return screen


/Users/baskauf/github/vandycite/gallery_works


In [5]:
images_frame = pd.read_csv(working_directory + '/image_list_dedup.csv')
wikidata_frame = pd.read_csv(working_directory + '/works_multiprop.csv') # generates an error saying that the columns have mixed types, but I don't see it
artstor_frame = pd.read_csv(working_directory + '/gallery_works_renamed1.csv')


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (100,101,103,104,106,107,108,118,119,122,124,125,126,128,129,130,133,134) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
wikidata_frame.set_index('inventory_number', inplace=True)
wikidata_frame. head()

,qid,label_en,description_en,inventory_number_uuid,inventory_number_collection,inventory_number_ref1_hash,inventory_number_ref1_statedIn,inventory_number_ref1_referenceUrl,inventory_number_ref1_retrieved_nodeId,inventory_number_ref1_retrieved_val,...,collection_ref1_retrieved_val,collection_ref1_retrieved_prec,location_uuid,location,location_ref1_hash,location_ref1_statedIn,location_ref1_referenceUrl,location_ref1_retrieved_nodeId,location_ref1_retrieved_val,location_ref1_retrieved_prec
inventory_number,,,,,,,,,,,,,,,,,,,,,
2017.001.062,Q102305506,The Christ of the Breadlines,print by Fritz Eichenberg,60965535-6F8D-4CB7-B702-D6A84E576DA3,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,NaN,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,87C848D7-1589-4F67-9DBE-12FE22CFF157,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,NaN,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0
1994.413,Q102315563,Seated female funerary figure,ceramics by Artist Unknown,9438F518-B646-4E12-AADD-D5FF50DF3346,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,NaN,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,83482B6C-AC70-46B8-AC76-2E65F1C42351,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,NaN,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0
1969.014,Q102315787,Head of a Young Girl or a Goddess,sculpture by Artist Unknown,B5F33FEE-A2B6-4D77-8764-C3942DC09E1B,Q18563658,09cfc74e2a883ae6ee704d256bb7fc57733abd97,NaN,https://library.artstor.org/#/asset/26753561,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,320CD258-94F0-420A-B386-EA2EEB98A3B6,Q18563658,09cfc74e2a883ae6ee704d256bb7fc57733abd97,NaN,https://library.artstor.org/#/asset/26753561,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0
2013.042,Q102949359,"'I Hate You For Hitting My Mother,’ Minneapolis",photograph by Donna Ferrato,AE86C1A8-48F0-4FD4-92D2-6DB10C878E24,Q18563658,c4d5f5b1f6a4c2840c84ddcb9500194882522c81,NaN,https://library.artstor.org/#/asset/26759879,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,0F670FF6-44C7-4243-B8E8-ECC5F493EF66,Q18563658,c4d5f5b1f6a4c2840c84ddcb9500194882522c81,NaN,https://library.artstor.org/#/asset/26759879,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0
1992.139 a-c,Q102958387,(An Imaginary) Night Attack in the War Between...,print by Yoshitora,B7A0A58E-8E8B-4B6A-A1E0-1EE854BA62B0,Q18563658,0f3872b67bf8edb2da2b2775249d865179be351d,NaN,https://library.artstor.org/#/asset/26756454,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2020-11-30T00:00:00Z,11,1F5DA01A-7D81-492D-8758-DEE5FD1226E4,Q18563658,0f3872b67bf8edb2da2b2775249d865179be351d,NaN,https://library.artstor.org/#/asset/26756454,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,11.0


In [7]:
images_frame.set_index('accession', inplace=True)
images_frame.head()

,has_image
accession,
1956.001,yes
1956.002,yes
1956.003,yes
1956.004,yes
1956.005,yes


In [8]:
artstor_frame.set_index('accession_number', inplace=True)
artstor_frame.head()

,ssid,filename,title,creator_string,date,classification,medium,measurements,style_period,country_culture,...,date_acquired,gift_of,purchased_from,credit_line,provenance,collection,last_change,notes,rights,media_url
accession_number,,,,,,,,,,,,,,,,,,,,,
2020.005,28884744,2020.005.tif,G2 Cover 11.06.2020,"Himid, Lubaina",2020,Prints,print on Hahnemühle Photorag Fine Art paper,12 x 10 in.,NaN,United Kingdom,...,2020,NaN,The Guardian Print Shop (Gallery Purchase),Collection of Vanderbilt University Fine Arts ...,NaN,NaN,NaN,NaN,IN COPYRIGHT,https://forum.jstor.org/assets/28884744/repres...
2020.003,28884738,NaN,Portrait of Julián Trujillo Largacha,"Mendoza, Pantaléon",late 19th century CE,Paintings,oil on canvas,NaN,NaN,Colombia,...,NaN,"Lisa Carr in honor of her father, Albert Trujillo",NaN,Collection of Vanderbilt University Fine Arts ...,Descended through donor's father's family.,NaN,NaN,NaN,NO KNOWN COPYRIGHT,NaN
2020.004,28884737,2020.004.jpg,Manhattan Skyline,"Gallagher, Sears (American painter, illustrato...",NaN,NaN,etching,12.25 x 9 in.,NaN,United States,...,2020,NaN,Wiscasset Bay Gallery (Fine Arts Gallery Purch...,NaN,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,COPYRIGHT NOT EVALUATED,https://forum.jstor.org/assets/28884737/repres...
2020.002,28589522,2020-002.jpg,Relics,"Woodruff, Hale Aspacio (American painter, 1900...",1935 CE,Graphic Arts,linocuts (prints),9 x 11.25 in,NaN,United States,...,2020,NaN,Purchased with The Joseph L. May History of Pr...,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,NaN,NaN,COPYRIGHT UNDETERMINED,https://forum.jstor.org/assets/28589522/repres...
2020.001,28262628,2020-001.tif,"The Chimneys, Portsmouth","Hassam, Childe (American painter and printmake...",1915 CE,Printmaking,etching,NaN,NaN,United States,...,2020,Purchased with Research Funds from the Andrew ...,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,NaN,NaN,NO KNOWN COPYRIGHT,https://forum.jstor.org/assets/28262628/repres...


# Remove duplicate column headers and join tables

Since all Artstor items are in Wikidata, an outer join should match everything.

In [9]:
# get rid of duplicate column names
wikidata_frame.drop('collection', axis=1, inplace=True)
wikidata_frame.drop('title', axis=1, inplace=True)

result = pd.concat([wikidata_frame, artstor_frame], axis=1, join="outer")
result.head()

,qid,label_en,description_en,inventory_number_uuid,inventory_number_collection,inventory_number_ref1_hash,inventory_number_ref1_statedIn,inventory_number_ref1_referenceUrl,inventory_number_ref1_retrieved_nodeId,inventory_number_ref1_retrieved_val,...,date_acquired,gift_of,purchased_from,credit_line,provenance,collection,last_change,notes,rights,media_url
2017.001.062,Q102305506,The Christ of the Breadlines,print by Fritz Eichenberg,60965535-6F8D-4CB7-B702-D6A84E576DA3,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,NaN,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2017,"Walter F. Schatz and Mary Pullig Schatz, MD",NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Mon Nov 26 00:00:00 EST 2018,Gift acquisition paperwork in file for 2017.00...,COPYRIGHT NOT EVALUATED,https://forum.jstor.org/assets/26760365/repres...
1994.413,Q102315563,Seated female funerary figure,ceramics by Artist Unknown,9438F518-B646-4E12-AADD-D5FF50DF3346,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,NaN,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,NaN,Marjorie and Leon Marlowe,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,The Marjorie and Leon Marlowe Collection,Mon Nov 12 00:00:00 EST 2018,#77\nDr. Headrick has three theories about the...,NaN,https://forum.jstor.org/assets/26757434/repres...
1969.014,Q102315787,Head of a Young Girl or a Goddess,sculpture by Artist Unknown,B5F33FEE-A2B6-4D77-8764-C3942DC09E1B,Q18563658,09cfc74e2a883ae6ee704d256bb7fc57733abd97,NaN,https://library.artstor.org/#/asset/26753561,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,1969,Mr. and Mrs. David K. Wilson,Mathias Komor,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Fri Oct 26 00:00:00 EDT 2018,Note: Broken off at neck (from a full figure)....,NO KNOWN COPYRIGHT,https://forum.jstor.org/assets/26753561/repres...
2013.042,Q102949359,"'I Hate You For Hitting My Mother,’ Minneapolis",photograph by Donna Ferrato,AE86C1A8-48F0-4FD4-92D2-6DB10C878E24,Q18563658,c4d5f5b1f6a4c2840c84ddcb9500194882522c81,NaN,https://library.artstor.org/#/asset/26759879,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2013,"Scott and Missy Tannen (both bachelor of arts,...",NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Mon Nov 26 00:00:00 EST 2018,"Image; sheet: 24"" x 20""\n\nThe acquisition of ...",IN COPYRIGHT,https://forum.jstor.org/assets/26759879/repres...
1992.139 a-c,Q102958387,(An Imaginary) Night Attack in the War Between...,print by Yoshitora,B7A0A58E-8E8B-4B6A-A1E0-1EE854BA62B0,Q18563658,0f3872b67bf8edb2da2b2775249d865179be351d,NaN,https://library.artstor.org/#/asset/26756454,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,1992,Herman D. Doochin,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,The Herman D. Doochin Collection,Wed Oct 03 00:00:00 EDT 2018,"A: Left panel; H. 14-1/4"" W.9-3/4"" ; artist's...",NaN,https://forum.jstor.org/assets/26756454/repres...


Do an inner join since we don't care about images not associated with items, nor items without images.

Basically, this just deletes works that don't have images from the earlier join.

In [10]:
result = pd.concat([images_frame, result], axis=1, join="inner")
# Don't need the has_image column any more
result.drop('has_image', axis=1, inplace=True)
works = result.sort_values(by=['qid'])
works.head()

,qid,label_en,description_en,inventory_number_uuid,inventory_number_collection,inventory_number_ref1_hash,inventory_number_ref1_statedIn,inventory_number_ref1_referenceUrl,inventory_number_ref1_retrieved_nodeId,inventory_number_ref1_retrieved_val,...,date_acquired,gift_of,purchased_from,credit_line,provenance,collection,last_change,notes,rights,media_url
2017.001.062,Q102305506,The Christ of the Breadlines,print by Fritz Eichenberg,60965535-6F8D-4CB7-B702-D6A84E576DA3,Q18563658,d1f8238d0b4bf5fadeb75381feceacfdd7f12af5,NaN,https://library.artstor.org/#/asset/26760365,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2017,"Walter F. Schatz and Mary Pullig Schatz, MD",NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Mon Nov 26 00:00:00 EST 2018,Gift acquisition paperwork in file for 2017.00...,COPYRIGHT NOT EVALUATED,https://forum.jstor.org/assets/26760365/repres...
1994.413,Q102315563,Seated female funerary figure,ceramics by Artist Unknown,9438F518-B646-4E12-AADD-D5FF50DF3346,Q18563658,7f7046f91b3d0ddd7d26cffcc9f1054db778fafa,NaN,https://library.artstor.org/#/asset/26757434,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,NaN,Marjorie and Leon Marlowe,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,The Marjorie and Leon Marlowe Collection,Mon Nov 12 00:00:00 EST 2018,#77\nDr. Headrick has three theories about the...,NaN,https://forum.jstor.org/assets/26757434/repres...
1992.139 a-c,Q102958387,(An Imaginary) Night Attack in the War Between...,print by Yoshitora,B7A0A58E-8E8B-4B6A-A1E0-1EE854BA62B0,Q18563658,0f3872b67bf8edb2da2b2775249d865179be351d,NaN,https://library.artstor.org/#/asset/26756454,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,1992,Herman D. Doochin,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,The Herman D. Doochin Collection,Wed Oct 03 00:00:00 EDT 2018,"A: Left panel; H. 14-1/4"" W.9-3/4"" ; artist's...",NaN,https://forum.jstor.org/assets/26756454/repres...
2016.002.01,Q102961140,#109,print by Bruce Conner,CCA2E88D-EB3F-4083-993E-A694035A3C8A,Q18563658,de2a6faedad59c4c8077dd14390161d2b190af84,NaN,https://library.artstor.org/#/asset/26760161,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2016,Conner Family Trust,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Tue Nov 29 00:00:00 EST 2016,From a series of offset lithographs made from ...,IN COPYRIGHT,https://forum.jstor.org/assets/26760161/repres...
2016.002.03,Q102961146,#116,print by Bruce Conner,A762EC6B-9E4C-49FE-94B1-47562347D122,Q18563658,dd49fd11eced4e9aecf9b06a7d02fdf6c4ea71bc,NaN,https://library.artstor.org/#/asset/26760163,0f364e2fcaf6b3e5870f19adb2cba00e,2020-11-30T00:00:00Z,...,2016,Conner Family Trust,NaN,Collection of Vanderbilt University Fine Arts ...,NaN,NaN,Tue Nov 29 00:00:00 EST 2016,From a series of offset lithographs made from ...,IN COPYRIGHT,https://forum.jstor.org/assets/26760163/repres...


In [14]:
works.to_csv(working_directory + '/items_with_images_full_metadata.csv')

In [11]:
headers = list(works.columns)
headers.sort()
headers

['classification',
 'collection',
 'collection_end_time_nodeId',
 'collection_end_time_prec',
 'collection_end_time_val',
 'collection_ref1_hash',
 'collection_ref1_referenceUrl',
 'collection_ref1_retrieved_nodeId',
 'collection_ref1_retrieved_prec',
 'collection_ref1_retrieved_val',
 'collection_ref1_statedIn',
 'collection_start_time_nodeId',
 'collection_start_time_prec',
 'collection_start_time_val',
 'collection_uuid',
 'copyright_status',
 'copyright_status_applies_to_jurisdiction',
 'copyright_status_determination_method',
 'copyright_status_ref1_hash',
 'copyright_status_ref1_referenceUrl',
 'copyright_status_ref1_retrieved_nodeId',
 'copyright_status_ref1_retrieved_prec',
 'copyright_status_ref1_retrieved_val',
 'copyright_status_ref1_statedIn',
 'copyright_status_uuid',
 'country',
 'country_culture',
 'country_of_origin',
 'country_of_origin_ref1_hash',
 'country_of_origin_ref1_referenceUrl',
 'country_of_origin_ref1_retrieved_nodeId',
 'country_of_origin_ref1_retrieved_pre

In [149]:
row = 580
#print(works.iloc[row])
#print(works.iloc[row]['inception_latest_date_val'])
for nothing in range(1):
    status = 'unknown'
    
    if work_too_old(copyright_applies_date, works.iloc[row]):
        status = 'inception prior to copyright cutoff'
        continue

    #print(works.iloc[row]['style_period'])
    if old_style_period(styles, works.iloc[row]):
        status = 'from style or period that ended prior to copyright cutoff'
        continue
        
    if not pd.isnull(works.iloc[row]['creator']):
        if works.iloc[row]['creator'] != 'Q4233718': # don't look up unknown artists
            if dead(deathDateLimit, works.iloc[row]['creator']):
                status = 'artist died before copyright cutoff'
                continue

            if person_too_old(birthDateLimit, works.iloc[row]['creator']):
                status = 'artist was born before '+ birthDateLimit
                continue

print(status)

This person died in  1891-09-14
*** Died too long ago !
artist died before copyright cutoff


In [ ]:
for index, work in works.iterrows():
    if work['term_localName']==table[row][localNameColumn] and vrow['version_status']=='recommended':
